In [2]:
import pandas as pd
import numpy as np

import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()

# nltk.download('wordnet')
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
# from collections import OrderedDict

import warnings
warnings.filterwarnings('ignore')

In [3]:
#TODO set up sqlalchemy to pull data in directly from mysql

tb_branded_products = pd.read_json('tb_branded_products.json')
tb_categories = pd.read_csv('tb_categories.csv')
tb_product_categories = pd.read_csv('tb_product_categories.csv')
ixone_products = pd.read_json('ixone_products.json')

tb_categories = tb_categories[['categories_id','categories_name','parent_categories_id']]

tb_df = tb_branded_products.merge(tb_product_categories, on ='product_id')
tb_df = tb_df.merge(tb_categories, on ='categories_id')

# Product Categorization

Objective is to develop an approach to categorize food products received from external food databases into a standardized category taxonomy.

Data sources include:
- Existing food dataset already categorized with a lengthy process including merging product description fields, excel deduping text strings, and an IF THEN ELSE sql query approach (~19k products)
- New food item dataset to be categorized (~50k products)

In [4]:
print('Existing dataset metrics:')
print('    Count of food items:', tb_df.count()[0])
print('    Count of parent categories:', tb_df['parent_categories_id'].nunique())
print('    Count of sub-categories:', tb_df['categories_id'].nunique())

Existing dataset metrics:
    Count of food items: 19210
    Count of parent categories: 9
    Count of sub-categories: 29


In [5]:
print('New dataset metrics:')
print('    Count of food items:', ixone_products.count()[0])

New dataset metrics:
    Count of food items: 53431


### Categories

- Inital problem is there is not yet a fully fleshed out taxonomy. The existing category list include 8 parent categories with 39 sub-categories. This is not a representative list of all food products that could be seen in external datasets. 
- As a result, in addition to categorizing products in new datasets, need to determine new categories.
- Note - primary interest is in categorizing shelf products, not products like beverages/dairy, which are included in the new datasets, so whether or not a priority, they do need to be handled
- Note - the existing categories have a lot of issues with puncutation, spelling, misleading terms..

In [6]:
print('Existing category metrics (exclude 1 for the catch-all category):')
print('    Count of unique categories:', tb_categories.count()[0])
print('    Count of parent categories:', tb_categories['parent_categories_id'].nunique())

Existing category metrics (exclude 1 for the catch-all category):
    Count of unique categories: 40
    Count of parent categories: 9


In [7]:
counts_df = tb_df.categories_name.value_counts().rename_axis('categories_name').reset_index(name='total_count')

df = tb_categories.merge(tb_categories, left_on='parent_categories_id', right_on='categories_id')
df = df[['categories_name_y', 'categories_id_y','categories_name_x','categories_id_x']]
df.columns = ['parent_category_name', 'parent_category_id','categories_name', 'category_id']
categories_df = df.merge(counts_df, on = 'categories_name', how='outer').fillna(0)
# df = df[df['parent_category_name'] != 'Main-category']

print('Category breakdown including total count of items currently categorized:')
categories_df
# df.to_csv('categories.csv')

Category breakdown including total count of items currently categorized:


,parent_category_name,parent_category_id,categories_name,category_id,total_count
0,Sauces,15,Pasta & Pizza Sauces,1,1952.0
1,Sauces,15,Cranberry Sauces,18,22.0
2,Sauces,15,Apple Sauces,19,912.0
3,Sauces,15,Simmer Sauce,40,42.0
4,Frozen Desserts,5,Ice Cream & Frozen Yoghurt,2,4400.0
5,Frozen Desserts,5,Other Frozen Desserts,6,207.0
6,Frozen Desserts,5,"Sorbet, Sherbet, Slushies",8,309.0
7,Frozen Desserts,5,Pops & Bars,9,1814.0
8,Main-category,0,Breakfast Cereals,3,0.0
9,Main-category,0,Frozen Desserts,5,0.0


## Approach Ideas

First need to clean up the Brand/Manufacturer fields, Product Description fields, Serving fields, and Ingredient fields. All food datasets are not the same, but generally should have fields in most of these areas.

### Idea 1
- Product categorization problems are typically approached as a supervised classification problem. Given there is already an existing categorized food dataset, can we use that along with the new dataset to train and test a model utilizing the above-mentioned fields?
- At this stage, this is not an option. The two datasets have limited overlap, there are only 13 matching food items according to the UPC codes. 
- Mapping between the columns in each and appending the two datasets doesn't seem that feasible. Don't have the same type of fields like brand or servings. Also the categorizations for existing categorized dataset are not great - not all categories are in there and some have very few food items.
- To create a classifier, need to label a lot more product data (potential helpers - AWS - MTurk, Ground Truth)

In [8]:
#identifier, brand, description, serving size, ingredients, category

#['product_id', 'product_description', 'brand_owners_id', 'serving_size', 'serving_size_unit',
#count_of_ingredients','product_ingredients_text', 'product_badges_text']
tb_branded_products.columns

Index(['product_id', 'product_expose', 'product_description', 'fdc_id',
       'gtin_upc', 'ean', 'brand_owners_id', 'household_serving_fulltext',
       'serving_size', 'serving_size_unit', 'image', 'product_url',
       'affiliate_url', 'data_source', 'modified_date', 'count_of_ingredients',
       'product_ingredients_text', 'product_badges_text',
       'product_allergens_text'],
      dtype='object')

In [9]:
# ['IXOneId', 'BrandName', 'ManufacturerNameHeader','ProductDescription', 'SecondaryProductDescription',
# 'IngredientsStatement']
ixone_products.columns

Index(['IXOneId', 'BrandName', 'Status', 'AllergensAncillary',
       'ProcessedOnEquipment', 'AttributesAncillary', 'Contains',
       'GlutenFreeYN', 'GMPYN', 'NonGMOYN', 'VeganYN', 'NonGMOCertifier',
       'InProduction', 'ManufacturerNameHeader', 'ProductDescription',
       'SecondaryProductDescription', 'IngredientsStatement', 'UPC11', 'UPC13',
       'Allergens', 'PlantBasedDerivedClaim', 'DoesNotContain', 'Process',
       'SugarSweetener', 'SustainabilityYN', 'ProductFlavor', 'ProductStyle',
       'MarketingClaims', 'SocialMediaAddresses', 'Website',
       'CountryOfOriginName', 'ManufacturerPhoneNumberPackaging', 'GrassFedYN',
       'KosherYN', 'CertificationsAncillary', 'OrganicYN',
       'KosherCertification', 'OrganicCertifyingBody', 'FreeOf', 'FairTradeYN',
       'FairTradeCertifier', 'CreationDate'],
      dtype='object')

In [10]:
df13 = ixone_products[~ixone_products['UPC13'].str.contains("[a-zA-Z]").fillna(False)]
df13['UPC13'] = df13['UPC13'].replace(r'^\s*$', np.nan, regex=True) 
df13 = df13.dropna(subset=['UPC13'])
df13['UPC13'] = df13['UPC13'].astype(str).astype(float).astype(int)

check = df13.merge(tb_df, left_on = 'UPC13', right_on = 'gtin_upc', how='inner')

print('Count of matched UPC codes between datasets:', check.count()[0])
check[['IXOneId', 'BrandName', 'ProductDescription',
       'UPC11', 'UPC13', 'gtin_upc','product_id','categories_id', 'categories_name']]

Count of matched UPC codes between datasets: 13


,IXOneId,BrandName,ProductDescription,UPC11,UPC13,gtin_upc,product_id,categories_id,categories_name
0,SNL210756,THINK!®,PROTEIN & FIBER HOT OATMEAL,75365671189,75365671189,75365671189,18887,13,Oats & Porridge
1,SNL202869,CLEVELAND KRAUT,SAUERKRAUT,86998200005,86998200005,86998200005,18939,42,"Pickles, Chutney, Kraut, Relish"
2,SNL5552,PROBAR®,MEAL BAR,85315210062,85315210062,85315210062,18143,4,"Snack, Energy & Granola Bars"
3,SNL154577,FAMILIA,CEREAL,07276201216,7276201216,7276201216,1446,12,Granola & Muesli
4,SNL189095,JUST,MAYO SPREAD,85966000411,85966000411,85966000411,18931,41,"Mustard, Mayo & Ketchup"
5,SNL189265,MICHELE'S GRANOLA®,GRANOLA,79357368270,79357368270,79357368270,3512,12,Granola & Muesli
6,SNL202868,CLEVELAND KRAUT,SAUERKRAUT,86998200003,86998200003,86998200003,18937,42,"Pickles, Chutney, Kraut, Relish"
7,SNL202866,CLEVELAND KRAUT,SAUERKRAUT,86998200001,86998200001,86998200001,18935,42,"Pickles, Chutney, Kraut, Relish"
8,SNL530256,CLEVELAND KITCHEN,GNARLY ASIAN ZING FERMENTED DRESSING,85977400720,85977400720,85977400720,18933,38,Dressing
9,SNL202864,CLEVELAND KRAUT,SAUERKRAUT,86998200002,86998200002,86998200002,18936,42,"Pickles, Chutney, Kraut, Relish"


### Alternative

- Need to clean the columns we care about to make them usable.
- Use bag of words/ngrams to identify most common words, update a food specific stop words and phrases.
- Clean the brand, descriptions, and ingredients - which would be best for mapping?
- Use descriptions - make a mapping to put into categories. Had to create new categories.
- Can also try out an unsupervised categorization. Use a clustering algorithim to identify clusters and manually determine their categories, potentially informed by existing dataset.

In [49]:
ixone_products = ixone_products[['IXOneId', 'BrandName', 'ManufacturerNameHeader', 'ProductDescription',
       'SecondaryProductDescription', 'IngredientsStatement']]
ixone_products = ixone_products.set_index('IXOneId')

ixone_products.head()

,BrandName,ManufacturerNameHeader,ProductDescription,SecondaryProductDescription,IngredientsStatement
IXOneId,,,,,
SNL334250,NOW®,NOW FOODS,BETTERSTEVIA LIQUID - MAPLE,SWEETENER,"VEGETABLE GLYCERIN, NATURAL MAPLE FLAVOR (IN A..."
SNL569248,UPCYCLED GRAIN PROJECT™,RUTHERFORD AND MEYER,SPENT GRAIN CRACKER,None,"SPENT BARLEY GRAIN MASH (43.5%), WHEAT FLOUR, ..."
SNL541603,SAKURA WAGYU FARMS,OHIO WAGYU BEEF LLC,HOT DOG UNCURED,AMERICAN STYLE KOBE,"WAGYU BEEF, WATER, CONTAINS 2% LESS SALT, SPIC..."
SNL525730,DELALLO®,DELALLO,PASTA FUSILLI ORG,None,ITALIAN ORGANIC DURUM WHEAT SEMOLINA.
SNL534756,RIEGA®,RIEGA FOODS,SEASONING TCO CARNE ASADA,None,"ORGANIC PAPRIKA, ORGANIC GARLIC, ORGANIC ONION..."


In [63]:
#CLEAN 

def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def remove_punctuation_exclude_commas(text):
    for punctuation in '!"#$%&\'()*+-./:;<=>?@[\\]^_`{|}~':
        text = text.replace(punctuation, '')
    return text

def remove_stop_phrases(text):
    stop_phrases= ["gluten(\s?\\.?\s?)free","extra(\s?\\.?\s?)virgin","cold(\s?\\.?\s?)pressed",
                  "shelf(\s?\\.?\s?)stable","raw(\s?\\.?\s?)&(\s?\\.?\s?)unfiltered",
                   "grain(\s?\\.?\s?)free", "plant(\s?\\.?\s?)based"
                  
                  ]
    for phrase in stop_phrases:
        text = re.sub(phrase, "", text, flags=re.IGNORECASE)
    return text


description_stop = ['% ', 'artesian', 'bag', 'big', 'blend', 'boutique', 'box', 'california', 'case', 'classic', 
                    'complete', 'craft', 'creative', 'crispy', 'ct', 'everything', 'fresh', 'gourmet', 'grassfed', 
                    'hard', 'heirloom', 'healthy','high', 'inch', 'italian', 'large', 'lb', 'liter', 'med', 'medium', 'microwave', 
                    'mid', 'mini', 'natural', 'naturally', 'novel', 'og', 'org', 'organic', 'original', 'oz', 
                    'pack', 'perfect', 'ph', 'pk', 'plain', 'plant-based', 'prebaked', 'premium', 'presliced', 
                    'quest', 'raw', 'refrigerated', 'roasted', 'serve', 'single', 'size', 'small', 'sparkling', 
                    'style', 'super', 'sweetened', 'tin', 'traditional', 'unbaked', 'uncured', 'vegan', 'whole',
                    'wonderful']

stop.extend(description_stop)


def clean_brand(data, cols, lowvol_limit):
    
    col_name = 'brand'
    df = data[cols]

    df[cols] = df[cols].astype(str)
#     df[col_name] = df[cols].agg(' '.join, axis=1) 
    df[col_name] = df[cols].apply(remove_punctuation)
    df[col_name] = df[col_name].str.replace('[^\x00-\x7F]','')
    df[col_name] = df[col_name].str.replace('\d+', '')
    df[col_name] = df[col_name].apply(lambda x: x.lower())

    value_counts = df[col_name].value_counts()
    df[col_name] = np.where(df[col_name].isin(value_counts.index[value_counts >= lowvol_limit]), df[col_name], 'low-volume')

    return df


def clean_description(data, cols, description_stop, lowvol_limit):
    
    col_name = 'description'
    df = data[cols]
    
    df[cols] = df[cols].astype(str)
#     df[col_name] = df[cols].agg(' '.join, axis=1)  
    df[col_name] = df[cols].apply(remove_punctuation)
    df[col_name] = df[col_name].str.replace('-',' ')
    df[col_name] = df[col_name].str.replace('[^\x00-\x7F]','')
    df[col_name] = df[col_name].str.replace('\d+', '')
    df[col_name] = df[col_name].apply(lambda x: x.lower())
    df[col_name] = df[col_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (description_stop)]))
    
    df[col_name] = df[col_name].apply(remove_stop_phrases)
    
    df[col_name]=df[col_name].apply(lambda x : filter(None,x.split(" ")))
    df[col_name]=df[col_name].apply(lambda x : [ps.stem(y) for y in x])
    df[col_name]=df[col_name].apply(lambda x : " ".join(x))
    
    df[col_name] = df[col_name].apply(lambda x: x.strip())

    value_counts = df[col_name].value_counts()
    df[col_name] = np.where(df[col_name].isin(value_counts.index[value_counts >= lowvol_limit]), df[col_name], 'low-volume')

#     df = pd.concat(data['IXOneId'], df[col_name])
#     df.index = df['IXOneId']
 

    return df


def clean_ingredients(data, cols, ingredients_stop, lowvol_limit):
    
    col_name = 'ingredients'
    df = data[cols]

    df[cols] = df[cols].astype(str)
    df[col_name] = df[cols].agg(' '.join, axis=1)  
    df[col_name] = df[col_name].str.replace(r"\(.*\)","")
    df[col_name] = df[col_name].apply(remove_punctuation_exclude_commas)
    df[col_name] = df[col_name].str.replace('[^\x00-\x7F]','')
    df[col_name] = df[col_name].str.replace('\d+', '')
    df[col_name] = df[col_name].apply(lambda x: x.lower())
    df[col_name] = df[col_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (ingredients_stop)]))

    df = df.reset_index()
    df = pd.concat([df[['IXOneId']], df[col_name].str.split(', ', n=4, expand=True)], axis=1)
  
    df.index = df['IXOneId']
    df = df[[0, 1, 2]]
    df.columns = ['ingredient1', 'ingredient2', 'ingredient3']

    value_counts1 = df['ingredient1'].value_counts()
    df['ingredient1'] = np.where(df['ingredient1'].isin(value_counts1.index[value_counts1 >= lowvol_limit]), df['ingredient1'], 'low-volume')

    value_counts2 = df['ingredient2'].value_counts()
    df['ingredient2'] = np.where(df['ingredient2'].isin(value_counts2.index[value_counts2 >= lowvol_limit]), df['ingredient2'], 'low-volume')

    value_counts3 = df['ingredient3'].value_counts()
    df['ingredient3'] = np.where(df['ingredient3'].isin(value_counts3.index[value_counts3 >= lowvol_limit]), df['ingredient3'], 'low-volume')

    return df

In [51]:
#REVIEW BRANDS

brand_cols = ['BrandName']
brand_df = clean_brand(ixone_products, brand_cols, 10)
brand_df.head()

# brand_df.count()
# brand_df.brand.nunique()
# brand_df.brand.value_counts().head(5)
# brand_df_dedup = brand_df[['brand']].drop_duplicates()
# brand_df_dedup.count()

,BrandName,brand
IXOneId,,
SNL334250,NOW®,now
SNL569248,UPCYCLED GRAIN PROJECT™,low-volume
SNL541603,SAKURA WAGYU FARMS,low-volume
SNL525730,DELALLO®,delallo
SNL534756,RIEGA®,riega


In [52]:
#could plot this to better select low volume limit

print('Percentage of items with brands if limit is 5:', round((53431-4675)/53431*100))
print('Percentage of items with brands if limit is 10:', round((53431-11647)/53431*100))
print('Percentage of items with brands if limit is 15:', round((53431-17533)/53431*100))
print('Percentage of items with brands if limit is 20:', round((53431-22550)/53431*100))

Percentage of items with brands if limit is 5: 91
Percentage of items with brands if limit is 10: 78
Percentage of items with brands if limit is 15: 67
Percentage of items with brands if limit is 20: 58


In [53]:
#REVIEW DESCRIPTIONS 

#identify common words add to stop words and phrases

In [54]:
description_cols = ['ProductDescription']
description_df = clean_description(ixone_products, description_cols, stop, 0)
description_df.head(2)

,ProductDescription,description
IXOneId,,
SNL334250,BETTERSTEVIA LIQUID - MAPLE,betterstevia liquid mapl
SNL569248,SPENT GRAIN CRACKER,spent grain cracker


In [55]:
# #Bag of Words - get most common words, and add back into phrases and stop words above
# CountVec = CountVectorizer(ngram_range=(1,1), stop_words='english')
# # CountVec = CountVectorizer(ngram_range=(2,2))

# Count_data = CountVec.fit_transform(description_df['description'])
# cv_df=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

# cv_df.loc['total']= cv_df.sum(numeric_only=True, axis=0)
# cv_df = cv_df[-1:].transpose().sort_values(by='total', ascending=False)
# cv_df.head(2)

In [18]:
## get out list of most common cleaning descriptions and create a mapping

# topdescriptions = description_df.description.value_counts()

# # topdescriptions.to_csv('topdescriptions.csv')
# topdescriptions.head()

In [19]:
#created mapping in excel, had to create some new categories 

mapping = pd.read_csv('description_map.csv')
mapping.head(2)

,description_substr,category
0,appl sauc,Apple Sauces
1,applesauc,Apple Sauces


In [20]:
# 1 if exact match to mapping, assign category
description_df = description_df.reset_index()
description_df = description_df.merge(mapping, left_on = 'description', right_on = 'description_substr', how = 'left')
description_df.count()

IXOneId               53436
ProductDescription    53436
description           53436
description_substr    13254
category              13254
dtype: int64

In [21]:
#2 if not exact match, but a substring matches, assign category (will have issues with duplicates)
description_dict = mapping.set_index('description_substr').T.to_dict('list')

getvals = '|'.join(r"\b{}\b".format(x) for x in description_dict.keys())
description_df['category_2'] = description_df['description'].str.extract('('+ getvals + ')', expand=False).map(description_dict)
description_df['category_2'] = description_df['category_2'].str[0]

description_df['clean_category'] = np.where(description_df['category'].isnull(), description_df['category_2'], description_df['category'])

In [24]:
print('Percentage categorized:', round(41095/53431,  3))
description_df.count()

Percentage categorized: 0.769


IXOneId               53436
ProductDescription    53436
description           53436
description_substr    13254
category              13254
category_2            41095
clean_category        41095
dtype: int64

In [25]:
check_null = description_df.loc[description_df.clean_category.isnull()]
# check_null.count()
# check_null.to_csv('review.csv')

In [26]:
#Bag of Words - get most common words, and add back into phrases and stop words above

# CountVec = CountVectorizer(ngram_range=(1,1), stop_words='english')
CountVec = CountVectorizer(ngram_range=(2,2))

Count_data = CountVec.fit_transform(check_null['description'])
cv_df=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())

cv_df.loc['total']= cv_df.sum(numeric_only=True, axis=0)
cv_df = cv_df[-1:].transpose().sort_values(by='total', ascending=False)
cv_df.head(10)

,total
sweet potato,41
monk fruit,41
clearli craft,37
half half,32
green chile,31
curri past,29
sweet drop,26
stevia sweeten,26
appleg nat,26
roll oat,23


In [27]:
ixone_counts = description_df.clean_category.value_counts().rename_axis('categories_name').reset_index(name='ixoneid_total_count')
combine = categories_df.merge(ixone_counts, on = 'categories_name', how='outer')
# combine
# combine.to_csv('final_categories.csv')

In [28]:
description_df=description_df[['IXOneId','clean_category']]
ixone_products = ixone_products.reset_index()

In [29]:
data = ixone_products.merge(description_df, on = 'IXOneId', how='outer')
data.columns = ['IXOneId', 'BrandName', 'ManufacturerNameHeader', 'ProductDescription',
       'SecondaryProductDescription', 'IngredientsStatement',
       'category']
# data.to_csv('ixoneid_w_categories.csv')

In [ ]:
#unused code

#     #remove duplicate words
#     df[col_name] = df[col_name].str.replace(r'\b(\w+)(\s+\1)+\b', r'\1')
#     df[col_name] = (df[col_name].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))


#     #lemmatization (skip stemming)

# def lemmatize_text(text):
#     w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#     lemmatizer = nltk.stem.WordNetLemmatizer()
#     return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
#     df[col_name] = df[col_name].apply(lemmatize_text)

# df[col] = df[col].apply(lambda x: word_tokenize(x))


    #print distribution of value counts of cols
#     print('Distribution of value counts of', col_name, 
#           df.groupby(col_name).count().quantile([0,0.25, 0.50, 0.75,1]))


#     #lemmatization (skip stemming) - FIX THIS IF USING
#     df[col_name] = df[col_name].apply(lemmatize_text)

    #tokenize - split into list of words with seperate columns (just for ingredients)
#     df[col_name]= df[col_name].apply(nltk.word_tokenize)
#      df = pd.concat([df[[0]], df[col_name].str.split(', ', n=4, expand=True)], axis=1)



In [64]:
ingredients_stop = ['carbonated', 'sparkling', 'pure', 'unbleached','whole','organic', 'certified', 'natural', 'and', "contains", "less", "cane", "sea", "filtered", "enriched", "purified"]

ingredient_cols = ['IngredientsStatement']

ingredient_df = clean_ingredients(ixone_products, ingredient_cols, ingredients_stop, 20)

In [65]:
ingredient_df.head()

,ingredient1,ingredient2,ingredient3
IXOneId,,,
SNL334250,low-volume,low-volume,stevia leaf extract
SNL569248,low-volume,low-volume,low-volume
SNL541603,low-volume,water,salt
SNL525730,low-volume,low-volume,low-volume
SNL534756,paprika,garlic,onion


In [61]:
brand_df = brand_df[['brand']]

description_df = clean_description(ixone_products, description_cols, stop, 0)
# description_df = description_df[['IXOneId','description']]

In [66]:
df = pd.concat([brand_df, ingredient_df, description_df], axis=1)

In [67]:
print(df.brand.nunique())
print(df.brand.value_counts().head(10))

1528
low-volume               11647
bob's red mill             341
amy's                      275
chef pierre                272
lundberg family farms      255
delallo                    251
kind                       249
organic valley             229
woodstock                  199
field day                  196
Name: brand, dtype: int64


In [68]:
print(df.description.nunique())
print(df.description.value_counts().head(10))

26323
cooki          474
ice cream      431
juic           394
protein bar    380
chocol bar     263
cracker        225
kombucha       223
granola        195
water          189
salsa          182
Name: description, dtype: int64


In [69]:
print(df.ingredient1.nunique())
print(df.ingredient1.value_counts().head(10))

364
low-volume      19629
water            5607
sugar            1867
                 1282
wheat flour      1176
flour             603
almonds           575
milk              478
pork              425
wheat flour       403
Name: ingredient1, dtype: int64


In [70]:
print(df.ingredient2.nunique())
print(df.ingredient2.value_counts().head(10))

319
low-volume       22869
water             3851
sugar             3839
salt              2113
sunflower oil      584
wheat flour        436
cream              395
flavors            363
almonds            329
honey              321
Name: ingredient2, dtype: int64


In [71]:
print(df.ingredient3.nunique())
print(df.ingredient3.value_counts().head(10))

325
low-volume       26068
salt              3044
sugar             2478
water             1264
citric acid        605
flavors            511
sunflower oil      446
cocoa butter       346
flavor             334
honey              328
Name: ingredient3, dtype: int64


## Try out unsupervised clustering

In [72]:
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn import feature_selection, linear_model
from sklearn import cluster, datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from kmodes.kmodes import KModes

In [82]:
# onehot = encoder.fit_transform(df['brand'])
test = pd.get_dummies(df, columns = ['brand', 'ingredient1', 'ingredient2', 'ingredient3', 'description'])

In [83]:
test.drop('ProductDescription', inplace=True, axis=1)

In [84]:
test.head()

,brand_ acres the seed co,brand_ b.c. kombucha,brand_ southwestern,brand_a l'olivier,brand_abe's,brand_absolutely gluten free,brand_acai roots,brand_acid league,brand_acme ice cream,brand_adirondack,...,description_zucchini pound cake,description_zucchini spiral,description_zucchini squash noodl,description_zucchini summer squash noodl,description_zucchini veggicinn,description_zwieback,description_zyn immun & recoveri drink lemon ginger flavor ( fl oz),description_zyn immun & recoveri drink mango lyche flavor ( fl oz),description_zyn immun & recoveri drink mix berri flavor ( fl oz),description_zyn immun & recoveri drink pomegran cranberri flavor ( fl oz)
IXOneId,,,,,,,,,,,,,,,,,,,,,
SNL334250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SNL569248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SNL541603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SNL525730,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SNL534756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# drop some
pca = PCA(.9)
pca.fit(test)
test2 = pca.transform(test)

In [ ]:
print(test.shape)
print(test.shape)

In [ ]:
kmeans = KMeans(50)
kmeans.fit(test2)

identified_clusters = kmeans.fit_predict(test2)
identified_clusters

In [ ]:
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=20, c='red')
plt.show()

In [ ]:
wcss=[]
for i in range(50,60,2):
    kmeans = KMeans(i)
    kmeans.fit(test2)
    wcss_iter = kmeans.inertia_
    wcss.append(wcss_iter)

number_clusters = range(50,60,2)
plt.plot(number_clusters,wcss)
plt.title('The Elbow title')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

### try kmodes so don't need dummy fields

In [ ]:
# Elbow curve to find optimal K
cost = []
K = range(60,70)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(df)
    cost.append(kmode.cost_)
    
plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
kmode = KModes(n_clusters=68, init = "random", n_init = 5)
clusters = kmode.fit_predict(df)
clusters

In [ ]:
df.insert(0, "Cluster", clusters, True)
df.head(20)

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/06/kmodes-clustering-algorithm-for-categorical-data/
#then apply that fit approach to the original dataset, 
# figure out how the categories join up with their sub categories and update it

### Could similarities be useful?

In [ ]:
### Another idea is a word2vec appraoch in interms of similarirty between the food words within the ingredients
# could train it on larger usda website 

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
a = nlp('BETTERSTEVIA LIQUID - MAPLE')
b = nlp('SPENT GRAIN CRACKER')
c = nlp('HOT DOG UNCURED')
d = nlp('PASTA FUSILLI ORG')
e = nlp('SEASONING TCO CARNE ASADA')

print(a.similarity(a))
print(a.similarity(b))
print(a.similarity(c))
print(a.similarity(d))
print(a.similarity(e))